# Gestion des métadonnées des espèces

Nous scannons les dossiers disponibles afin d'en faire un dataframe et réutiliser ces informations.
 Puis nous récupérons les métadonnées depuis l'API Mistral grâce à un prompt optimisé (optimisation du grounding, du prompt engineering)
 un sleep de 3s a été ajouté afin d'éviter de trop spam l'API

In [6]:
import pandas as pd
import os
from dotenv import load_dotenv
from mistralai import Mistral
import time

In [7]:
folder_all_animals = [d for d in os.listdir("data/OpenAnimalTracks/raw_imgs") if os.path.isdir(os.path.join("data/OpenAnimalTracks/raw_imgs", d))]
df_all_animals = pd.DataFrame(folder_all_animals, columns=["Nom du dossier"])

# Charger les variables d'environnement
load_dotenv()

# Nombre total d'animaux à scanner
total_animaux = len(df_all_animals)
reste_a_scanner = total_animaux

api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans les variables d'environnement.")

model = "open-mistral-nemo"

client = Mistral(api_key=api_key)

fichier_csv = "data/csv/metadata.csv"

# Supprimer le fichier s'il existe déjà
if os.path.exists(fichier_csv):
    os.remove(fichier_csv)

donnees_animaux = []

for index, animal in enumerate(df_all_animals["Nom du dossier"]):

    reste_a_scanner = total_animaux - (index + 1)

    print(f"🔍 Recherche des informations pour {animal}...")
    print(f"Il reste {reste_a_scanner} animaux à scanner")

    prompt = f"""
    En français, donne-moi les informations suivantes sur {animal} :
    - le nom de l'espèce,
    - la famille,
    - le nom latin,
    - la population estimée (uniquement un nombre, sans texte, sans unité, sans ponctuation sauf le point ou la virgule pour les milliers),
    - la localisation (uniquement le ou les pays, séparés par un espace).
    - la description, une courte phrase décrivant l'animal.

    Attention :
    - Ne mets pas d'explication ou de phrase, uniquement les valeurs demandées.
    - Pour la population, écris uniquement un nombre sans texte. Par exemple : 1000000 au lieu de '1 million d'espèces environ'.
    - Pour la localisation, écris uniquement le ou les pays séparés par un espace.
    - Pour la Description, je souhaite 30 mots grand maximum.
    - Le nom de l'espèce sera systématiquement traduit en français

    Présente les informations sous ce format exact :
    Espèce : <nom de l'espèce>
    Famille : <famille>
    Nom latin : <nom latin>
    Description: <description>
    Population estimée : <population estimée>
    Localisation : <localisation>
    """

    try:
        chat_response = client.chat.complete(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )

        # Extraire la réponse
        reponse = chat_response.choices[0].message.content

        # Parser la réponse pour extraire les valeurs
        informations = {}
        for ligne in reponse.split("\n"):
            if ":" in ligne:
                cle, valeur = ligne.split(":", 1)
                informations[cle.strip()] = valeur.strip()

        # Ajouter les informations à la liste
        donnees_animaux.append(informations)

    except Exception as e:
        print(f"Erreur lors de la récupération des infos pour {animal} : {e}")

    # Pause pour éviter d'être banni
    print("⏳ Attente de 3 secondes avant la prochaine requête...")
    time.sleep(3)

# Création du DataFrame
df_animaux = pd.DataFrame(donnees_animaux)

display(df_animaux)

# Sauvegarde dans un CSV
df_animaux.to_csv(fichier_csv, index=False)

print(f"Les informations des animaux ont été enregistrées dans {fichier_csv}.")


🔍 Recherche des informations pour blackbear...
Il reste 17 animaux à scanner
⏳ Attente de 3 secondes avant la prochaine requête...
🔍 Recherche des informations pour mouse...
Il reste 16 animaux à scanner
⏳ Attente de 3 secondes avant la prochaine requête...
🔍 Recherche des informations pour horse...
Il reste 15 animaux à scanner
⏳ Attente de 3 secondes avant la prochaine requête...
🔍 Recherche des informations pour gray_fox...
Il reste 14 animaux à scanner
⏳ Attente de 3 secondes avant la prochaine requête...
🔍 Recherche des informations pour coyote...
Il reste 13 animaux à scanner
⏳ Attente de 3 secondes avant la prochaine requête...
🔍 Recherche des informations pour skunk...
Il reste 12 animaux à scanner
⏳ Attente de 3 secondes avant la prochaine requête...
🔍 Recherche des informations pour american_mink...
Il reste 11 animaux à scanner
⏳ Attente de 3 secondes avant la prochaine requête...
🔍 Recherche des informations pour mountainlion...
Il reste 10 animaux à scanner
⏳ Attente de 3 

,Espèce,Famille,Nom latin,Description,Population estimée,Localisation
0,Ours noir,Ursidae,Ursus americanus,"L'ours noir est un ours de taille moyenne, au ...",900000,Canada États-Unis Mexique
1,Souris domestique,Muridés,Mus musculus,"Petite souris grise aux yeux rouges, vivant so...",700000000,Monde entier
2,Cheval,Équidés,Equus ferus caballus,"Mammifère domestiqué, connu pour son utilisati...",58000000,"France, États-Unis, Chine, Russie, Brésil"
3,Renard gris,Canidés,Urocyon cinereoargenteus,Le renard gris est un petit canidé au pelage g...,4000000,États-Unis Mexique
4,Coyote,Canidés,Canis latrans,"Le coyote est un canidé de taille moyenne, au ...",3000000,États-Unis Canada Mexique
5,Skunk,Mustelidae,Mephitis mephitis,Le skunk est un petit mammifère carnivore conn...,3000000,États-Unis Canada
6,Vison d'Amérique,Mustelidae,Neogale vison,Petit mustélidé semi-aquatique au pelage brun-...,1000000,Canada États-Unis
7,Puma concolor,Felidae,Puma concolor,"Grand félin de couleur brun-jaune, avec une qu...",300000,"Amérique du Nord, Amérique centrale, Amérique ..."
8,Eider à duvet,Anatidae,Somateria mollissima,"Canard marin au plumage blanc et noir, avec un...",1500000,"Canada, Russie, Norvège, Islande, Groenland"
9,Rat domestique,Muridés,Rattus norvegicus,Petit mammifère nocturnal aux longues griffes ...,600000000,"France, Chine, Inde, États-Unis"


Les informations des animaux ont été enregistrées dans data/csv/metadata.csv.
